In [7]:
import Tkinter, tkFileDialog, os
from Libs import LVQLib as lvqlib
from Libs import ExpWriter as ew

In [24]:
class LVQUI(Tkinter.Tk):    
    def __init__(self, parent):
        self.viewdex = 0        # current view index
        self.curView = []
        self.parent  = parent
        self.state   = { 'savePath': None, 'vectors': [], 'network': None } # Model
        self.views   = [ 'Load Vectors', 'Label Neurons', 'Train & Save' ]
        self.inits   = [ self.loadViewInit, self.labelViewInit, self.trainViewInit ]
        
        Tkinter.Tk.__init__(self, parent)
        self.initialize()
        
    def initialize(self):
        self.grid()
        self.grid_columnconfigure(0, weight=2)
        self.resizable(False, True)
        
        self.screen_title = Tkinter.Label( self, text=self.views[self.viewdex], 
                                           anchor='center')
        self.forwardBtn   = Tkinter.Button(self, text=" Next  >>>", command=self.nextView,
                                           width=10)
        self.backwardBtn  = Tkinter.Button(self, text="<<<  Back ", command=self.prevView,
                                           width=10)
        # Grid Positioning
        self.screen_title.grid(column=4, row=1, columnspan=4, sticky='W', padx=20)
        self.forwardBtn.grid(column=16,  row=1, columnspan=4, sticky='E', padx=40)
        self.backwardBtn.grid(column=0,  row=1, columnspan=4, sticky='W', padx=40)
        
        self.loadViewInit()
        self.update()
        
        self.geometry(self.geometry()) # comment out for auto resizing.
        
    def getFileDialogue(self):
        """
        Open a file chooser and extract the path 
        (add to filepaths for input vectors).
        """
        filey = tkFileDialog.askopenfile(parent=self, mode='r', 
                                         title="Audio Sample")
        if (filey != None):
            self.addFile(filey.name)
            self.listBox.insert('end', self.state['vectors'][-1])
            filey.close()
            #print self.state['vectors'] # Seems to have fixed the original problem
            #self.loadViewInit()
            
        self.update()
        
    def nextView(self):
        if (self.viewdex < len(self.views) - 1):
            self.clearView()
            self.viewdex += 1
            self.inits[self.viewdex]()
            self.screen_title['text'] = self.views[self.viewdex]
            
        self.update()
        
    def prevView(self):
        if (self.viewdex > 0):
            self.clearView()
            self.viewdex -= 1
            self.inits[self.viewdex]()
            self.screen_title['text'] = self.views[self.viewdex]

        self.update()
    
    def clearView(self):
        """
        Clear the canvas from all non-permanent elements
        """
        for i in self.curView:
            i.grid_forget()
            
        self.update()
        self.curView = []
        
    def addFile(self, filepath):
        self.state['vectors'].append(filepath)
        return self.state['vectors']
        
    def setSavePath(self, filepath):
        self.state['savePath'] = filepath
        return self.state['savePath']
        
    def initializeNetwork(self, name, vSize, outs):
        self.state['network'] = lvqlib.LVQNet(ins, outs)
        return self.state['network']
        
    def initializeVectors(self, network, vectors):
        """
        Vectors are filepaths to unique category drum sounds
        """
        for vector in vectors:
            network.enterCSV(vector)
        
    def saveNetwork(self, net, path):
        f = ew.SEWriter("Test", "./")
        
        for neuron in net:
            f.writeSexpArr(neuron)
            
        f.finish()
        
    """
    
    Views:
    
        1) Load in samples
        2) Label Classifications -- show the network
        3) Training & save screen
    
    """
    def loadViewInit(self):
        """
        Load the view (1): 
            Load samples for training
            (and vector initialization of the neurons)
        """
        self.fileLabel = Tkinter.Label(self,  text="Files Imported:")
        self.listBox   = Tkinter.Listbox(self, width=75)
        self.fileBtn   = Tkinter.Button(self, text="Add Vector",    
                                        command=self.getFileDialogue)
        
        self.curView = [ self.fileLabel, self.fileBtn, self.listBox ]
        
        self.curView[0].grid(column=0, row=2, columnspan=1, sticky='N', pady=2)
        self.curView[1].grid(column=0, row=3, columnspan=1, sticky='N', pady=2)
        self.curView[2].grid(column=4, row=3, columnspan=6, sticky='W', pady=10)
        
        self.update()
        
    def labelViewInit(self):
        """
        Load the view (2): 
            Label neuron classifiers
        """
        v = Tkinter.StringVar()
        v.set("Label Name for Neuron")
        self.instruct = Tkinter.Label(self, textvariable=v)
        self.enterBox = Tkinter.Entry(self)
        
        self.curView = [ self.instruct, self.enterBox ]
        
        self.curView[0].grid(column=1, row=4, sticky='W')
        self.curView[1].grid(column=2, row=4, sticky='W')
        
    def trainViewInit(self):
        """
        Load the view (3):
            Training Parameters & Output / Progress
        """
        v = Tkinter.StringVar()
        self.filepath = Tkinter.Label(self,  textvariable=v)
        self.saveBtn  = Tkinter.Button(self, text="Save Network")
        
        self.curView = [ self.filepath, self.saveBtn ]
        
        self.curView[0].grid(column=2, row=4, columnspan=1, sticky='W')
        self.curView[1].grid(column=2, row=5, columnspan=1, sticky='W')

In [25]:
if __name__ == '__main__':
    gui = LVQUI(None)
    gui.title('LVQ Drum Training')
    gui.mainloop()